Import The Libraries📚

In [ ]:
import os
import pandas as pd
import torch
import json
# import torchaudio
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline
# import plotly.express as px
# import librosa
# import librosa.display
# import IPython.display as ipd
# import sklearn
# import warnings
# import seaborn as sns
# warnings.filterwarnings('ignore')

In [ ]:
import os
import ast
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import librosa.display
import IPython.display as display

In [ ]:
os.getcwd()

SEE THE DATA 🐔:

In [ ]:
path = '/kaggle/input/birdclef-2022/'
os.listdir(path)

train_meta = pd.read_csv(path+'train_metadata.csv')
test_data = pd.read_csv(path+'test.csv')
ebird_data = pd.read_csv(path+'eBird_Taxonomy_v2021.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

with open(path+'scored_birds.json') as f:
    scored_birds = json.load(f)

In [ ]:
test_data

In [ ]:
train_meta.head()

In [ ]:
train_meta.shape

EXAMPLE ON ROW 0

In [ ]:
row = 0
train_meta.iloc[row]

In [ ]:
label = train_meta.loc[row, 'primary_label']
filename = train_meta.loc[row, 'filename']

print(filename)
# Check if the file is in the folder
filename.split('/')[1] in os.listdir(path+'train_audio/'+label)

data, samplerate = sf.read(path+'train_audio/'+filename)
print('snipe of data:', data[:4])
print('samplerate:', samplerate)

In [ ]:
def plot_audio_file(data, samplerate):
    """ Plot the audio data"""
    
    sr = samplerate
    fig = plt.figure(figsize=(8, 4))
    x = range(len(data))
    y = data
    plt.plot(x, y)
    plt.plot(x, y, color='red')
    plt.legend(loc='upper center')
    plt.grid()
    

plot_audio_file(data, samplerate)

In [ ]:
def plot_spectrogram(data, samplerate):
    """ Plot spectrogram with mel scaling """
    
    sr = samplerate
    spectrogram = librosa.feature.melspectrogram(y=data, sr=sr)
    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    librosa.display.specshow(log_spectrogram, sr=sr, x_axis='time', y_axis='mel')
    
plot_spectrogram(data, samplerate)

In [ ]:
display.Audio(path+'train_audio/'+filename)

Exploratory Data Analysis 
There are 152 primary lables and 152 common names. Both are related. The labels are not evenly distributed.

In [ ]:
# train_meta['primary_label'].value_counts()[0:10]
len(set(list(train_meta['primary_label'])))

In [ ]:
train_meta['common_name'].value_counts()[0:10]

In [ ]:
sf.read(path+'train_audio/'+filename)

**TEST MODEL**

In [ ]:
import csv

In [ ]:
# Create empty csv file 
header = "length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var label".split()

file = open('data.csv', 'w', newline = '')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [ ]:
# Transform each .wav file into a .csv row:
for line in train_meta.index:
    sound_name = path+'train_audio/'+train_meta['filename'][line]
    y, sr = librosa.load(sound_name, mono = True, duration = 30)
    chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
    rmse = librosa.feature.rms(y = y)
    spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
    spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
    rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y = y, sr = sr)
    to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

    for e in mfcc:
        to_append += f' {np.mean(e)}'
        
    to_append += str(" ") + str(train_meta['primary_label'][line])
    file = open('data.csv', 'a', newline = '')
    
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

In [ ]:
df = pd.read_csv('data.csv')
# df

2. Features extraction
In the preprocessing of the data, feature extraction is necessary before running the training. The purpose is to define the inputs and outputs of the neural network.

OUTPUT (y): last column which is the label.
You cannot use text directly for training. You will encode these labels with the LabelEncoder() function of sklearn.preprocessing.

Before running a model, you need to convert this type of categorical text data into numerical data that the model can understand.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
class_list = df.iloc[:,-1]
converter = LabelEncoder()
y = converter.fit_transform(class_list)
print("y: ", y)

INPUTS (X): all other columns are input parameters to the neural network.
Remove the first column which does not provide any information for the training (the filename) and the last one which corresponds to the output.

In [ ]:
fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:, 1:26], dtype=float))

Split dataset for training

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Building the model
The first step is to build the model and display the summary.

For the CNN model, all hidden layers use a ReLU activation function, the output layer a Softmax function and a Dropout is used to avoid overfitting.

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation = 'relu', input_shape = (X.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(152, activation = 'softmax'),
])

print(model.summary())

In [ ]:
# model.fit(X_train, y_train, epochs=100, verbose=False, shuffle=False)
# y_pred = model.predict(X_train)
# mse_krm=mean_squared_error(y_train, y_pred)

In [ ]:
# def trainModel(model,epochs, optimizer):
#     batch_size = 128
#     model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
#     return model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = epochs, batch_size = batch_size)

In [ ]:
# model_history = trainModel(model = model, epochs = 100, optimizer = 'adam')

In [ ]:
# loss_train_curve = model_history.history["loss"]
# loss_val_curve = model_history.history["val_loss"]
# plt.plot(loss_train_curve, label = "Train")
# plt.plot(loss_val_curve, label = "Validation")
# plt.legend(loc = 'upper right')
# plt.title("Loss")
# plt.show()

In [ ]:
# test_loss, test_acc = model.evaluate(X_test, y_test, batch_size = 128)
# print("The test loss is: ", test_loss)
# print("The best accuracy is: ", test_acc*100)

In [ ]:
from sklearn.metrics import mean_squared_error
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(X, y,  epochs = 100, batch_size = 128)
y_pred  = model.predict(X)
# mse_krm=mean_squared_error(y_test, y_krm)

In [ ]:
# generate argmax for predictions
classes = np.argmax(y_pred, axis = 1)
# print(classes)

In [ ]:
# transform classes number into classes name
result = converter.inverse_transform(classes)
print(result)

In [ ]:
model.save("modelv1-birdclef.h5")

**Format submission : un fichier + 5 sec + un oiseau --> TRUE or FALSE**

In [ ]:
# soundscapes = glob.glob(os.path.join(PATH_DATASET, "test_soundscapes", "*.ogg"))
# soundscapes = list(map(os.path.basename, soundscapes))
# converted = []
# for batch in Parallel(n_jobs=3)(delayed(_convert_and_export)(fn) for fn in tqdm(soundscapes)):
#     converted += batch
# # _= list(map(_convert_and_export, tqdm(train_meta["filename"])))
# df_converted = pd.DataFrame(converted)
# display(df_converted.head())

In [ ]:
# 453028782.ogg
test_data

**PREPARE TEST DATA**

In [ ]:
# # Create empty csv file 
# header = "filename length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var label".split()

# file = open('datatest.csv', 'w', newline = '')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(header)
    
# # Transform each .wav file into a .csv row:
# for line in test_data.index:
#     sound_name = path+'test_soundscapes/'+test_data['file_id'][line]
#     y, sr = librosa.load(sound_name, mono = True, duration = 30)
#     chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
#     rmse = librosa.feature.rms(y = y)
#     spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
#     spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
#     rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
#     zcr = librosa.feature.zero_crossing_rate(y)
#     mfcc = librosa.feature.mfcc(y = y, sr = sr)
#     to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

#     for e in mfcc:
#         to_append += f' {np.mean(e)}'
        
#     to_append += str(" ") + str(test_data['bird'][line])
#     file = open('datatest.csv', 'a', newline = '')
    
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())    
    
# df = pd.read_csv('datatest.csv')

In [ ]:
# Create empty csv file 
header = "filename length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var".split()

file = open('datatest.csv', 'w', newline = '')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

files = sorted(os.listdir(path+'test_soundscapes/'))

with open('../input/birdclef-2022/scored_birds.json') as fp:
    SCORED_BIRDS = json.load(fp)

In [ ]:
import math

In [ ]:
def create_df_test_from_path():
    files = sorted(os.listdir(path+'test_soundscapes/'))
    data = []
    for f in files:
        wv, sr = librosa.load(path+'test_soundscapes/' + f)
        n_chunks = math.ceil(len(wv) / sr / 5)
        filename = f
        row_prefix = f[:-4]
#         bird = SCORED_BIRDS[0]
        for chunk in range(1, n_chunks + 1):
            for bird in SCORED_BIRDS:
                row_id = f"{f[:-4]}_{bird}_{chunk*5}"
            
                ending_second = chunk*5
                data.append((filename, row_prefix, ending_second, bird))
            
    return  pd.DataFrame(data, columns=['filename', 'row_prefix', 'ending_second', 'birds'])
        
test_df = create_df_test_from_path()

In [ ]:
test_df

ICI UNE PREDICTION PAR FICHIER ET PAS PAR 5SEC --> IL FAUT INTEGRER LA POSSIBILIT2 QU4IL N4Y AI AUCUN OISEAU

In [ ]:
# Transform each .wav file into a .csv row:
# for line in test_df.index:
#     sound_name = path+'test_soundscapes/'+test_df['filename'][line]
for f in files:
    sound_name = path+'test_soundscapes/'+f
    print(sound_name)
    y, sr = librosa.load(sound_name, mono = True, duration = 60)
    chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
    rmse = librosa.feature.rms(y = y)
    spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
    spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
    rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y = y, sr = sr)
    to_append = f'{f} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

    for e in mfcc:
        to_append += f' {np.mean(e)}'
        
#     to_append += str(" ") + str(test_data['bird'][line])
    file = open('datatest.csv', 'a', newline = '')
    
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())    
    
df = pd.read_csv('datatest.csv')

In [ ]:
df

In [ ]:
# class_list = df.iloc[:,-1]
# converter = LabelEncoder()
# y = converter.fit_transform(class_list)

In [ ]:
from sklearn.preprocessing import StandardScaler

fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:, 1:26], dtype=float))
X

In [ ]:
y_pred  = model.predict(X)
classes = np.argmax(y_pred, axis = 1)

In [ ]:
# transform classes number into classes name
result = converter.inverse_transform(classes)
print(result)

In [ ]:
# insert preditcion is the dataframe
df.insert(0, 'result', result)

In [ ]:
test_df = pd.merge(test_df, df[['filename','result']], how='left', left_on = 'filename', right_on = 'filename')

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
#     print(test_df['birds'][i])
#     print(test_df['result'][i])
#     preds = 1 if test_df['birds'][i] == test_df['result'][i] else 0
#     print(preds)

In [ ]:

# import json
# from tqdm.auto import tqdm

# #Liste des oiseaux a predire
# with open(path+'scored_birds.json') as f:
#     scored_birds = json.load(f)

# print(scored_birds)

# submission = []
# for i, row in tqdm(test_data.iterrows(), total=len(test_data)):
#     preds = True if test_data['bird'][i] == result[i] else False
#     for bird in scored_birds:
#         submission.append({
#             "row_id": f"{row['file_id']}_{bird}_{row['end_time']}",
#             "target": preds#[bird] > 1. / len(model.labels),
#         })

# df_submission = pd.DataFrame(submission).set_index("row_id")
# df_submission.to_csv("submission.csv")


In [ ]:
test_df

In [ ]:

import json

submission = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    preds = True if test_df['birds'][i] == test_df['result'][i] else False
    submission.append({
        "row_id": f"{row['row_prefix']}_{row['birds']}_{row['ending_second']}",
        "target": preds#[bird] > 1. / len(model.labels),
    })

df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")


In [ ]:
# train_csv=pd.read_csv('../input/birdclef-2022/train_metadata.csv')
# train_csv.head()

Loading and Visualizing an audio file:
librosa.load: loads an audio file as a floating point time series and gives it's native sampling rate.
The sampling frequency (or sample rate) is the number of samples (data points) per second in an audio.
We can check the audio length by dividing the total number of data points by the sampling frequency.

In [ ]:
# y, sr = librosa.load(subfly)
# print('y:', y, '\n')
# print('y shape:', np.shape(y), '\n')
# print('Sample Rate (KHz):', sr, '\n')
# print('Check Len of Audio:', np.shape(y)[0]/sr)

SPECTROGRAM

In [ ]:
# n_fft=2048
# hop_length=512
# # Short-time Fourier transform (STFT)
# D_astfly = np.abs(librosa.stft(audio_astfly, n_fft = n_fft, hop_length = hop_length))
# # Convert an amplitude spectrogram to Decibels-scaled spectrogram.
# DB_astfly = librosa.amplitude_to_db(D_astfly, ref = np.max)
# # === PLOT ===
# fig, ax = plt.subplots(1, 1, figsize=(12, 6))
# fig.suptitle('Log Frequency Spectrogram', fontsize=16)
# # fig.delaxes(ax[1, 2])
# img=librosa.display.specshow(DB_astfly, sr = sr_astfly, hop_length = hop_length, x_axis = 'time', 
#                          y_axis = 'log', cmap = 'cool', ax=ax)
# ax.set_title('ASTFLY', fontsize=13) 
# plt.colorbar(img,ax=ax)

Mel-frequency cepstral coefficients (MFCCs)
One popular audio feature extraction method is the Mel-frequency cepstral coefficients (MFCC), which has 39 features. The feature count is small enough to force the model to learn the information of the audio. 12 parameters are related to the amplitude of frequencies. It models the characteristics of the human voice. The extraction flow of MFCC features is depicted below:

In [ ]:
# mfcc=librosa.feature.mfcc(y=audio_astfly, sr=sr_astfly)
# fig, ax = plt.subplots(1,figsize = (12, 6))
# img = librosa.display.specshow(mfcc, x_axis='time', ax=ax)
# print(mfcc.shape)
# fig.colorbar(img, ax=ax)
# ax.set(title='MFCC')

SUBMISSION

In [ ]:
# class CFG:
#     ######################
#     # Globals #
#     ######################
#     seed = 1213

#     ######################
#     # Data #
#     ######################
#     train_datadir = Path("../input/birdclef-2021/train_short_audio")
#     train_csv = "../input/birdclef-2021/train_metadata.csv"
#     train_soundscape = "../input/birdclef-2021/train_soundscape_labels.csv"

#     ######################
#     # Dataset #
#     ######################
#     transforms = {
#         "train": [{"name": "Normalize"}],
#         "valid": [{"name": "Normalize"}],
#         "test": [{"name": "Normalize"}]
#     }
#     period = 30
#     n_mels = 128
    
#     sample_rate = 32000
#     target_columns = [
#         "afrsil1",
#         "akekee",
#         "akepa1",
#         "akiapo",
#         "akikik",
#         "amewig",
#         "aniani",
#         "apapan",
#         "arcter",
#         "barpet",
#         "bcnher",
#         "belkin1",
#         "bkbplo",
#         "bknsti",
#         "bkwpet",
#         "blkfra",
#         "blknod",
#         "bongul",
#         "brant",
#         "brnboo",
#         "brnnod",
#         "brnowl",
#         "brtcur",
#         "bubsan",
#         "buffle",
#         "bulpet",
#         "burpar",
#         "buwtea",
#         "cacgoo1",
#         "calqua",
#         "cangoo",
#         "canvas",
#         "caster1",
#         "categr",
#         "chbsan",
#         "chemun",
#         "chukar",
#         "cintea",
#         "comgal1",
#         "commyn",
#         "compea",
#         "comsan",
#         "comwax",
#         "coopet",
#         "crehon",
#         "dunlin",
#         "elepai",
#         "ercfra",
#         "eurwig",
#         "fragul",
#         "gadwal",
#         "gamqua",
#         "glwgul",
#         "gnwtea",
#         "golphe",
#         "grbher3",
#         "grefri",
#         "gresca",
#         "gryfra",
#         "gwfgoo",
#         "hawama",
#         "hawcoo",
#         "hawcre",
#         "hawgoo",
#         "hawhaw",
#         "hawpet1",
#         "hoomer",
#         "houfin",
#         "houspa",
#         "hudgod",
#         "iiwi",
#         "incter1",
#         "jabwar",
#         "japqua",
#         "kalphe",
#         "kauama",
#         "laugul",
#         "layalb",
#         "lcspet",
#         "leasan",
#         "leater1",
#         "lessca",
#         "lesyel",
#         "lobdow",
#         "lotjae",
#         "madpet",
#         "magpet1",
#         "mallar3",
#         "masboo",
#         "mauala",
#         "maupar",
#         "merlin",
#         "mitpar",
#         "moudov",
#         "norcar",
#         "norhar2",
#         "normoc",
#         "norpin",
#         "norsho",
#         "nutman",
#         "oahama",
#         "omao",
#         "osprey",
#         "pagplo",
#         "palila",
#         "parjae",
#         "pecsan",
#         "peflov",
#         "perfal",
#         "pibgre",
#         "pomjae",
#         "puaioh",
#         "reccar",
#         "redava",
#         "redjun",
#         "redpha1",
#         "refboo",
#         "rempar",
#         "rettro",
#         "ribgul",
#         "rinduc",
#         "rinphe",
#         "rocpig",
#         "rorpar",
#         "rudtur",
#         "ruff",
#         "saffin",
#         "sander",
#         "semplo",
#         "sheowl",
#         "shtsan",
#         "skylar",
#         "snogoo",
#         "sooshe",
#         "sooter1",
#         "sopsku1",
#         "sora",
#         "spodov",
#         "sposan",
#         "towsol",
#         "wantat1",
#         "warwhe1",
#         "wesmea",
#         "wessan",
#         "wetshe",
#         "whfibi",
#         "whiter",
#         "whttro",
#         "wiltur",
#         "yebcar",
#         "yefcan",
#         "zebdov",
#     ]
#     bird2id = {b: i for i, b in enumerate(target_columns)}
#     id2bird = {i: b for i, b in enumerate(target_columns)}
#     scored_birds = ["akiapo", "aniani", "apapan", "barpet", "crehon", "elepai", "ercfra", "hawama", "hawcre", "hawgoo", "hawhaw", "hawpet1", "houfin", "iiwi", "jabwar", "maupar", "omao", "puaioh", "skylar", "warwhe1", "yefcan"]
#     loader_params = {
#         "train": {
#             "batch_size": 64,
#             "num_workers": 20,
#             "shuffle": True
#         },
#         "valid": {
#             "batch_size": 64,
#             "num_workers": 20,
#             "shuffle": False
#         },
#         "test": {
#             "batch_size": 64,
#             "num_workers": 20,
#             "shuffle": False
#         }
#     }
#     models_cfg = [{"resnest26d": "../input/birdclef2022-weights/resnest26d/fold0/birdclef_2022/10p941rk/checkpoints/best_f1.ckpt",}]
#     num_classes = len(target_columns)
# print(f"model_num: {len(CFG.models_cfg)}")

In [ ]:
# TARGET_SR = 32000
# DATADIR = Path("../input/birdclef-2022/test_soundscapes/")

In [ ]:
# all_audios = list(DATADIR.glob("*.ogg"))
# sample_submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')
# sample_submission